In [64]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px

# Load data
import WeisWSClient
from WeisWSClient.api.scientific_objects_api import ScientificObjectsApi  
from WeisWSClient.api.data_api import DataApi  
from WeisWSClient.api.brapiv1token_api import Brapiv1tokenApi  
from WeisWSClient.api.variables_api import VariablesApi
from WeisWSClient.rest import ApiException
from dash.dependencies import Input, Output

pythonClient = WeisWSClient.ApiClient()
pythonClient.connect_to_opensilex_ws(username="admin@opensilex.org", password="admin")

var_api = WeisWSClient.VariablesApi(pythonClient)
vars_name = var_api.get_variables_by_search()
page_size_var = vars_name['metadata']['pagination']['totalCount']
vars_name = var_api.get_variables_by_search(page_size = page_size_var)
vars_name = vars_name['result'][0:len(vars_name['result'])]
df_vars = pd.DataFrame([o.__dict__ for o in vars_name])

data_api = WeisWSClient.DataApi(pythonClient)

external_stylesheets = ['/assets/style.css']
# Initialize the app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

def get_options(lab,uri):
    dict_list = []
    for i, j  in zip(lab, uri):
        dict_list.append({'label': i, 'value': j})
    return dict_list


app.layout = html.Div(
    children=[
        html.Div(className='row',
                children=[
                html.Div(className='four columns div-user-controls',
                     children=[
                         html.H2('DASH - MAP VISUALIZATION'),
                         html.P('Visualising map from model simulation - Dash.'),
                         html.P('Pick a variable and level of depth below.'),
                         html.Div(
                             className='div-for-dropdown',
                             children=[
                                 dcc.Dropdown(id='stockselector', 
                                              options=get_options(df_vars["_label"][81:], df_vars["_uri"][81:]),
                                              multi=False, value=df_vars["_uri"][81],
                                              style={'backgroundColor': '#1E1E1E'},
                                              className='stockselector'
                                              ),
                             ],
                             style={'color': '#1E1E1E'}),
                         html.Div([html.Div([
            dcc.Slider(
                    id = "niv",
                    updatemode = "drag",
                    vertical = False,
                    marks = {i: "Level {}".format(i) for i in [1,2,3,4,5]},
                    min = 1,
                    max = 5,
                    step = 1,
                    value = 1)], style = {"width": "300px"}),
                    ],
            style = {"width": "2%", "height":"100%","display":"inline-block","title":"trwhg"}),
            
                        ]
                     ),
                    html.Div(className='eight columns div-for-charts bg-grey',
                             children=[
                                 dcc.Loading(dcc.Graph(id='timeseries'))
                             ], style={'display': 'inline-block', 'float': 'left','vertical-align': 'middle!important'})
                              ])
        ]

)

# Callback for timeseries price
@app.callback(Output('timeseries', 'figure'),
        [Input('stockselector', 'value'),
        Input('niv','value')
        ])
def plot(choice,niv):
    obj_api = WeisWSClient.ScientificObjectsApi(pythonClient)
    objs_name = obj_api.get_scientific_objects_by_search(alias = "LV"+str(niv), page_size = 1)
    page_size_obj = objs_name['metadata']['pagination']['totalCount']
    objs_name = obj_api.get_scientific_objects_by_search(alias = "LV"+str(niv), page_size = page_size_obj)
    objs_name = objs_name['result'][0:len(objs_name['result'])]
    df_objs = pd.DataFrame([o.__dict__ for o in objs_name])
    objs = df_objs[['_label','_uri','_geometry']]
    objs.columns = ['lab','ScientificObjectURI','Geom']
    
    dt = data_api.get_data(choice, async_req=True, page_size = 29005)
    ds = dt.get()
    data_date = []
    for i in ds['result']:
        data_date.append(i._date)
    data_obj = []
    for i in ds['result']:
        data_obj.append(i.object_uri)
    data_value = []
    for i in ds['result']:
        data_value.append(i.value)
    data_variable = []
    for i in ds['result']:
        data_variable.append(i.variable_uri)

    ds = pd.DataFrame(data_date, columns = ['date'])
    ds['obj'] = data_obj
    ds['val'] = data_value
    ds['var'] = data_variable
    ds['dateStr'] = ds['date'].dt.strftime('%d/%m/%y %H:%M')
    ds = ds.sort_values(by="dateStr")
    dataj = ds.merge(objs, left_on = "obj", right_on = "ScientificObjectURI", how = "left")
    dataj["lab"] = dataj["lab"].astype('str')
    dataj["Geom"] = dataj["Geom"].astype('str')
    dataj['coord'] = dataj['Geom'].apply(lambda s: s[s.find("[")+1:s.find("]")])
    dataj['lat'] = dataj['coord'].apply(lambda x: x.split(",", 1)[1] if len(x.split(",", 1))==2 else 'NaN')
    dataj['lon'] = dataj['coord'].apply(lambda x: x.split(",", 1)[0] if len(x.split(",", 1))==2 else 'NaN')
    dataj["lat"] = dataj['lat'].apply(lambda x: float(x))
    dataj["lon"] = dataj['lon'].apply(lambda x: float(x))
    dataj['val'] = dataj['val'].apply(lambda x: (float("NaN")) if x =="" else float(x))
    
    fig = px.scatter_mapbox(dataj, lat="lat", lon="lon",animation_frame="dateStr",
                            color="val", zoom=8)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig
   
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run